Link to repo - common to all files: 'https://github.com/Derick047/PC3/tree/main/PC3-DataSets'

**Importacion de Archivos**

In [64]:
import pandas as pd

In [65]:
# all the links
education ='https://raw.githubusercontent.com/Derick047/PC3/refs/heads/main/PC3-DataSets/Education%20expenditures.csv'
electricity = 'https://raw.githubusercontent.com/Derick047/PC3/refs/heads/main/PC3-DataSets/Electricity%20-%20installed%20generating%20capacity.csv'
energy = 'https://raw.githubusercontent.com/Derick047/PC3/refs/heads/main/PC3-DataSets/Energy%20consumption%20per%20capita.csv'
reserves = 'https://raw.githubusercontent.com/Derick047/PC3/refs/heads/main/PC3-DataSets/Reserves%20of%20foreign%20exchange%20and%20gold.csv'

In [66]:
# Convirtiendo a DF y Limpieza de filas
education = pd.read_csv(education)
electricity = pd.read_csv(electricity)
energy = pd.read_csv(energy)
reserves = pd.read_csv(reserves)

**Limpieza**

In [42]:
# Limpiando nombres de columnas
for df in [education, electricity, energy, reserves]:
  df.columns = df.columns.str.strip()

In [44]:
education.columns

Index(['name', 'slug', '% of GDP', 'date_of_information', 'ranking', 'region'], dtype='object')

In [49]:
electricity.columns

Index(['name', 'slug', 'kW', 'date_of_information', 'ranking', 'region'], dtype='object')

In [50]:
energy.columns

Index(['name', 'slug', 'Btu/person', 'date_of_information', 'ranking',
       'region'],
      dtype='object')

In [51]:
reserves.columns

Index(['name', 'slug', 'value', 'date_of_information', 'ranking', 'region'], dtype='object')

In [45]:
# Educación
education_clean = education[['name', 'region', '% of GDP']].rename(
    columns={'name': 'Country', 'region': 'Region', '% of GDP': 'Education_GDP%'})

In [46]:
# Electricidad
electricity_clean = electricity[['name', 'region', 'kW']].rename(
    columns={'name': 'Country', 'region': 'Region', 'kW': 'Electricity_kW'})
electricity_clean['Electricity_kW'] = electricity_clean['Electricity_kW'].str.replace(",", "").astype(float)

In [47]:
# Energía
energy_clean = energy[['name', 'region', 'Btu/person']].rename(
    columns={'name': 'Country', 'region': 'Region', 'Btu/person': 'Energy_BtuPerCapita'})
energy_clean['Energy_BtuPerCapita'] = energy_clean['Energy_BtuPerCapita'].str.replace(",", "").astype(float)

In [48]:
# Reservas
reserves_clean = reserves[['name', 'region', 'value']].rename(
    columns={'name': 'Country', 'region': 'Region', 'value': 'NaturalReserves_ValueUSD'})
reserves_clean['NaturalReserves_ValueUSD'] = reserves_clean['NaturalReserves_ValueUSD'].str.replace("[$,]", "", regex=True).astype(float)

**Uniendo los Archivos**

In [53]:
df = education_clean.merge(electricity_clean[['Country', 'Electricity_kW']], on='Country')
df = df.merge(energy_clean[['Country', 'Energy_BtuPerCapita']], on='Country')
df = df.merge(reserves_clean[['Country', 'NaturalReserves_ValueUSD']], on='Country')

**Creando Columnas de Intervalos**

In [60]:
#Se dividio las columnas en 5 intervalos con igual cantidad de datos
etiquetas = ['Muy bajo', 'Bajo', 'Medio', 'Alto', 'Muy alto']
for col in ['Education_GDP%', 'Electricity_kW', 'Energy_BtuPerCapita', 'NaturalReserves_ValueUSD']:
    df[f'{col}_Interval'] = pd.qcut(df[col], 5, labels=etiquetas)

In [58]:
stats_by_region = df.groupby('Region')[['Education_GDP%', 'Electricity_kW', 'Energy_BtuPerCapita', 'NaturalReserves_ValueUSD']].agg(['mean', 'min', 'max'])

**Resultados**

In [59]:
display(df)

,Country,Region,Education_GDP%,Electricity_kW,Energy_BtuPerCapita,NaturalReserves_ValueUSD,Education_GDP%_Interval,Electricity_kW_Interval,Energy_BtuPerCapita_Interval,NaturalReserves_ValueUSD_Interval
0,Solomon Islands,Australia and Oceania,12.8,37000.0,5655000.0,6.882200e+08,Muy alto,Muy bajo,Muy bajo,Muy bajo
1,Bolivia,South America,9.8,4375000.0,29340000.0,8.710000e+07,Muy alto,Medio,Medio,Muy bajo
2,Namibia,Africa,9.6,646000.0,21734000.0,3.356000e+09,Muy alto,Bajo,Bajo,Bajo
3,Sierra Leone,Africa,9.1,149000.0,2301000.0,4.956990e+08,Muy alto,Muy bajo,Muy bajo,Muy bajo
4,Belize,Central America and the Caribbean,8.7,220000.0,30752000.0,4.980870e+08,Muy alto,Muy bajo,Medio,Muy bajo
...,...,...,...,...,...,...,...,...,...,...
171,Haiti,Central America and the Caribbean,1.4,472000.0,3486000.0,2.566000e+09,Muy bajo,Bajo,Muy bajo,Bajo
172,Papua New Guinea,East and Southeast Asia,1.4,1148000.0,8781000.0,3.901000e+09,Muy bajo,Bajo,Bajo,Medio
173,Venezuela,South America,1.3,33493000.0,54474000.0,9.794000e+09,Muy bajo,Muy alto,Medio,Medio
174,Nigeria,Africa,0.5,4094000.0,7993000.0,3.861200e+10,Muy bajo,Medio,Muy bajo,Alto


In [62]:
#Estadisticas por Region
display(stats_by_region)

Education_GDP%            Electricity_kW  \
                                            mean  min   max           mean   
Region                                                                       
Africa                                  4.268000  0.3   9.6   4.619820e+06   
Australia and Oceania                   6.314286  2.2  12.8   1.706100e+07   
Central America and the Caribbean       4.555000  1.4   8.7   1.580500e+06   
Central Asia                            4.333333  2.8   6.2   4.239578e+07   
East and Southeast Asia                 3.633333  1.4   6.3   2.108446e+08   
Europe                                  5.151429  3.1   7.2   4.027509e+07   
Middle East                             4.528571  1.7   7.8   3.381729e+07   
North America                           5.200000  4.3   6.1   5.008580e+08   
South America                           4.808333  1.3   9.8   3.550408e+07   
South Asia                              3.850000  1.9   7.0   7.211612e+07   

                                                              \
                                           min           max   
Region                                                         
Africa                                 29000.0  6.598900e+07   
Australia and Oceania                  34000.0  1.081930e+08   
Central America and the Caribbean      41000.0  6.581000e+06   
Central Asia                         3944000.0  3.019260e+08   
East and Southeast Asia               277000.0  2.949000e+09   
Europe                                779000.0  2.756580e+08   
Middle East                           352000.0  1.196200e+08   
North America                      105586000.0  1.235000e+09   
South America                         259000.0  2.402510e+08   
South Asia                            432000.0  4.991360e+08   

                                  Energy_BtuPerCapita              \
                                                 mean         min   
Region                                                              
Africa                                   1.412284e+07    649000.0   
Australia and Oceania                    6.221671e+07   5655000.0   
Central America and the Caribbean        5.587300e+07   3486000.0   
Central Asia                             1.039161e+08  16192000.0   
East and Southeast Asia                  1.186874e+08   6825000.0   
Europe                                   1.131422e+08  27407000.0   
Middle East                              2.406509e+08  14991000.0   
North America                            2.158707e+08  57539000.0   
South America                            4.641467e+07  25733000.0   
South Asia                               2.350638e+07   3380000.0   

                                               NaturalReserves_ValueUSD  \
                                           max                     mean   
Region                                                                    
Africa                              98847000.0             6.860587e+09   
Australia and Oceania              223158000.0             1.147904e+10   
Central America and the Caribbean  153952000.0             4.561870e+09   
Central Asia                       261142000.0             7.555822e+10   
East and Southeast Asia            643259000.0             3.536142e+11   
Europe                             234698000.0             6.415875e+10   
Middle East                        814308000.0             9.432479e+10   
North America                      311599000.0             1.890850e+11   
South America                       78496000.0             4.748692e+10   
South Asia                          64082000.0             7.907593e+10   

                                                               
                                            min           max  
Region                                                         
Africa                             3.045000e+07  6.844800e+10  
Australia and Oceania              3.965300e+08  5.44550

**Guardamos los Archivos**

In [63]:
df.to_csv("final_result.csv", index=False)
stats_by_region.to_csv("stats_by_region.csv")